In [1]:
import sys
from typing import Literal

import numpy as np
import pandas as pd
import plotly.figure_factory as ff
import plotly.graph_objects as go
import tensorflow as tf
import tensorflow_probability as tfp
from plotly.colors import DEFAULT_PLOTLY_COLORS as colors
from tensorflow.keras.layers import GRU, Dense, Embedding, SimpleRNN, StringLookup
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop

%load_ext autoreload
%autoreload 2

sys.path.append("../")
from equation_discover import *

2023-12-29 22:53:49.377628: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [41]:
X = pd.DataFrame(np.linspace(-2 * np.pi, 2 * np.pi), columns=["var_x"])
y = np.sin((X * 2 + 1).squeeze())

sampler = RNNSampler(BASE_TOKENS, 16, 1)
regressor = SymbolicRegressor(sampler, SymbolicLoss(), n_samples=12)
optimizer = RMSprop(learning_rate=1e-3)
x = pandas_to_tensor(X)
y = pandas_to_tensor(y)

root - DEBUG - 29-Dec-23 23:04:40 - Initial call, object=RNNSampler
root - DEBUG - 29-Dec-23 23:04:41 - Probs before constraints, object=RNNSampler, func=call, probs=<Tensor shape: (1, 10)
[[0.11, 0.09, ..., 0.05, 0.10]]>
root - DEBUG - 29-Dec-23 23:04:41 - After constraints, object=RNNSampler, func=call, probs=<Tensor shape: (1, 10)
[[0.11, 0.09, ..., 0.00, 0.00]]>
root - DEBUG - 29-Dec-23 23:04:41 - After normalization, object=RNNSampler, func=call, probs=<Tensor shape: (1, 10)
[[0.13, 0.10, ..., 0.00, 0.00]]>


In [42]:
def train_step(regressor: SymbolicRegressor, x, y):
    with tf.GradientTape() as tape:
        outputs = regressor({"X": x, "y": y})
        loss = regressor.loss_func(y, outputs)
    outputs.rewards = rsquared(y, outputs.y)
    trainable_variables = regressor.sampler.variables
    grads = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(grads, trainable_variables))
    return outputs

In [43]:
rewards = []
for _ in range(10):
    results = train_step(regressor, x, y)
    if any([tf.math.is_nan(result) for result in results.rewards]):
        print('early break')
        break
    rewards.append(results.rewards)

root - DEBUG - 29-Dec-23 23:04:42 - Sampling token iteration 0, object=RNNSampler
root - DEBUG - 29-Dec-23 23:04:42 - Probs before constraints, object=RNNSampler, func=call, probs=<Tensor shape: (12, 10)
[[0.11, 0.09, ..., 0.05, 0.10]
 [0.11, 0.09, ..., 0.05, 0.10]
 ...
 [0.11, 0.09, ..., 0.05, 0.10]
 [0.11, 0.09, ..., 0.05, 0.10]]>
root - DEBUG - 29-Dec-23 23:04:42 - After constraints, object=RNNSampler, func=call, probs=<Tensor shape: (12, 10)
[[0.11, 0.09, ..., 0.00, 0.00]
 [0.11, 0.09, ..., 0.00, 0.00]
 ...
 [0.11, 0.09, ..., 0.00, 0.00]
 [0.11, 0.09, ..., 0.00, 0.00]]>
root - DEBUG - 29-Dec-23 23:04:42 - After normalization, object=RNNSampler, func=call, probs=<Tensor shape: (12, 10)
[[0.13, 0.10, ..., 0.00, 0.00]
 [0.13, 0.10, ..., 0.00, 0.00]
 ...
 [0.13, 0.10, ..., 0.00, 0.00]
 [0.13, 0.10, ..., 0.00, 0.00]]>
root - DEBUG - 29-Dec-23 23:04:42 - Sampling token iteration 1, object=RNNSampler
root - DEBUG - 29-Dec-23 23:04:42 - Probs before constraints, object=RNNSampler, func=cal

In [44]:
rewards[0]

<tf.Tensor: shape=(12,), dtype=float32, numpy=
array([0.58211213, 0.01164853, 0.        , 0.32887483, 0.6278294 ,
       0.5896761 , 0.6418639 , 0.46759215, 0.05056982, 0.55055565,
       0.54626274, 0.48419815], dtype=float32)>

In [22]:
expression = results.expressions[4]

In [ ]:
expression.tree.left_child.left_child.left_child.left_child.left_child.left_child

In [ ]:
def plot_expresion(expression, x):
    trace = go.Scatter(x=x["var_x"], y=expression.eval(x).numpy())
    return go.Figure(
        [
            trace,
            go.Scatter(
                x=x["var_x"], y=y, line=dict(dash="dash", color=trace.marker.color)
            ),
        ]
    )

In [ ]:
plot_expresion(expression, x)

In [ ]:
expression.eval(X)